In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import scipy as stats
import pandas as pd
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics as mt
from pprint import pprint
import matplotlib as plt

In [2]:
df_ask = pd.read_csv('bittrex-bitcoin-pairs/BTC_PAIRS_ASK.csv')
df_bid = pd.read_csv('bittrex-bitcoin-pairs/BTC_PAIRS_BID.csv')
df_price = pd.read_csv('bittrex-bitcoin-pairs/BTC_PAIRS_PRICE.csv')
df_vol = pd.read_csv('bittrex-bitcoin-pairs/BTC_PAIRS_VOLUME.csv')

In [3]:
ask_time_col = df_ask.columns[0]
bid_time_col = df_bid.columns[0]
price_time_col = df_price.columns[0]
vol_time_col = df_vol.columns[0]

df_ask[ask_time_col] = pd.to_datetime(df_ask[ask_time_col], unit = 's', utc = True).apply(lambda x: x.date().strftime('%m-%d-%Y'))

df_bid[bid_time_col] = pd.to_datetime(df_bid[bid_time_col], unit = 's', utc = True).apply(lambda x: x.date().strftime('%m-%d-%Y'))

df_price[price_time_col] = pd.to_datetime(df_price[price_time_col], unit = 's', utc = True).apply(lambda x: x.date().strftime('%m-%d-%Y'))

df_vol[vol_time_col] = pd.to_datetime(df_vol[vol_time_col], unit = 's', utc = True).apply(lambda x: x.date().strftime('%m-%d-%Y'))

In [4]:
df_ask = df_ask.rename(columns ={ask_time_col:'Time'})
df_bid = df_bid.rename(columns ={bid_time_col:'Time'})
df_price = df_price.rename(columns ={price_time_col:'Time'})
df_vol = df_vol.rename(columns ={vol_time_col:'Time'})

In [5]:
ltc_df = df_price[['Time','BTC_LTC']]
eth_df = df_price[['Time','BTC_ETH']]
dash_df = df_price[['Time','BTC_DASH']]
xrp_df = df_price[['Time','BTC_XRP']]

In [6]:
t5_price = df_price[['BTC_LTC','BTC_ETH','BTC_ZEC','BTC_DASH','BTC_XRP']]
t5_price.iloc[2745] = (t5_price.iloc[2744]+t5_price.iloc[2746])/2




/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
t5_adj = t5_price.copy()
t5_adj.loc[-1] = [0, 0, 0, 0, 0]

In [8]:
t5_price.iloc[2745]

BTC_LTC     0.018771
BTC_ETH     0.063948
BTC_ZEC     0.028754
BTC_DASH    0.047050
BTC_XRP     0.000079
Name: 2745, dtype: float64

In [9]:
# (t5_price.iloc[2744][0]+t5_price.iloc[2746][0])/2

In [10]:
t5_adj.index = t5_adj.index + 1
t5_adj.sort_index(inplace = True)
t5_adj.head()



,BTC_LTC,BTC_ETH,BTC_ZEC,BTC_DASH,BTC_XRP
0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.019582,0.072955,0.027883,0.049795,0.000081
2,0.019650,0.072960,0.027900,0.049790,0.000081
3,0.019650,0.072660,0.027900,0.049830,0.000081
4,0.019650,0.072600,0.027900,0.049830,0.000081


In [11]:
diff_df = t5_price - t5_adj

diff_df = diff_df.drop(index = 40162)
diff_df = diff_df.drop(index = 0)
diff_df.index = diff_df.index - 1




In [12]:
diff_df.head()

,BTC_LTC,BTC_ETH,BTC_ZEC,BTC_DASH,BTC_XRP
0,6.800000e-05,0.000005,0.000017,-0.000005,-6.000000e-08
1,0.000000e+00,-0.000300,0.000000,0.000040,1.000000e-08
2,0.000000e+00,-0.000060,0.000000,0.000000,2.900000e-07
3,-6.800000e-05,0.000000,0.000000,-0.000494,-2.900000e-07
4,-1.000000e-06,0.000000,0.000000,0.000000,2.900000e-07


In [13]:
t5_price_division = t5_price.copy()
t5_price_division = t5_price_division.drop(index = 40161)


rel_df = diff_df/t5_price_division

In [14]:
rel_df.describe()

,BTC_LTC,BTC_ETH,BTC_ZEC,BTC_DASH,BTC_XRP
count,40161.000000,4.016100e+04,4.016100e+04,40161.000000,40161.000000
mean,-0.000002,-6.205566e-07,3.548589e-06,0.000002,0.000002
std,0.002690,2.463382e-03,2.318534e-03,0.002596,0.002131
min,-0.014260,-2.073029e-02,-1.353941e-02,-0.016868,-0.018481
25%,-0.000350,-6.256236e-04,-3.697108e-07,0.000000,-0.000286
50%,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
75%,0.000305,6.058220e-04,0.000000e+00,0.000000,0.000280
max,0.023335,1.448137e-02,1.585688e-02,0.021362,0.022048


In [15]:
# for x in rel_df:
#     print(rel_df[x])

In [16]:
# rel_df.head()

In [17]:
rel_df.to_csv('no_time.csv')

In [18]:
rel_df['Date'] = df_ask['Time']
rel_df.set_index('Date', inplace = True)


In [19]:
rel_df.to_csv('with_time.csv')